# <center> Lab 3-5: Send J1939 Multi Frame Messages
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center><img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" />
### <center> Instructor: Dr. Jeremy Daily<br>Fall 2020

## Prerequisites

Successfully run [02_SendCAN_RX.ipynb].

Be sure can-utils is installed:
```
sudo apt install can-utils
```

In [1]:
#!/usr/bin/python3
import socket
import struct

# Open a socket and bind to it from SocketCAN
sock = socket.socket(socket.PF_CAN, socket.SOCK_RAW, socket.CAN_RAW)
interface = "vcan3"

# Bind to the interface
sock.bind((interface,))

# To match this data structure, the following struct format can be used:
can_frame_format = "<lB3x8s"

In [4]:
# Define a send single J1939 frame function:
# Define the CAN Arbitration ID
def send_j1939_frame(pgn,sa,can_data,da=0xFF,priority=6):
    PF = (pgn & 0xFF00) >> 8
    if PF < 240:
        PS = da
    else:
        PS = (pgn & 0x00FF)
    
    can_id = priority << 26    
    can_id += (pgn & 0x3FF00) << 8
    can_id += PS << 8
    can_id += sa
    print("{:08X}".format(can_id))
    
    #Set the extended frame format bit.
    can_id |= socket.CAN_EFF_FLAG
    #The data length code cannot be larger than 8
    can_dlc = min(len(can_data),8)
    
    #Pack the information into 16 bytes
    can_packet = struct.pack(can_frame_format, can_id, can_dlc, can_data )
    
    #Send out the CAN frame
    sock.send(can_packet)

In [5]:
for i in range(10):
    pgn = 0xffff
    sa = 0xff
    can_data = b'Jeremy'
    
    send_j1939_frame(pgn,sa,can_data)

18FFFFFF
18FFFFFF
18FFFFFF
18FFFFFF
18FFFFFF
18FFFFFF
18FFFFFF
18FFFFFF
18FFFFFF
18FFFFFF


In [6]:
def sendBAM(pgn,sa,can_message,da=0xFF,priority=6):
    num_bytes = len(can_message)
    if num_bytes > 8: #BAM
        num_frames = (num_bytes//7)
        if num_bytes%7 > 0:
            num_frames +=1
        can_data = b'\x20' #bam
        can_data += struct.pack("<H",num_bytes)
        can_data += struct.pack("B",num_frames)
        can_data += b'\xFF' #Reserved Bit
        can_data += struct.pack("B",(pgn & 0x0000FF))
        can_data += struct.pack("B",(pgn & 0x00FF00) >> 8)
        can_data += struct.pack("B",(pgn & 0xFF0000) >> 16)
        print(can_data)
        send_j1939_frame(0xEC00,0xF9,can_data,da=0xFF,priority=6)
        for i in range(num_frames):
            can_data = struct.pack("B",i+1) + can_message[7*i:7*(i+1)]
            for i in range(len(can_data),8):
                can_data += b'\xff'
            send_j1939_frame(0xEB00,0xF9,can_data,da=0xFF,priority=6)
            print(can_data)

In [9]:
sendBAM(0xFFFF,0xF9,b'Vehicle Communication is still Awesome')

b' &\x00\x06\xff\xff\xff\x00'
18ECFFF9
18EBFFF9
b'\x01Vehicle'
18EBFFF9
b'\x02 Commun'
18EBFFF9
b'\x03ication'
18EBFFF9
b'\x04 is sti'
18EBFFF9
b'\x05ll Awes'
18EBFFF9
b'\x06ome\xff\xff\xff\xff'


Run Notebook [04_ParseJ1939_Multi_Frame.ipynb](04_ParseJ1939_Multi_Frame.ipynb) to see the messages arrive and decode.

## In Lab Assignment:
Send another student or your instructor your first and last name over vcan3. 